# How much does the image position vary due to hexapod motions?

Craig Lage  19-Aug-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, calcNextDay
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from astropy.time import Time, TimeDelta
import yaml, copy
import pandas as pd

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
def calculateHexMotion(camShifts, m2Shifts):
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    camCoefs = [camHexXY, camHexXY, 0, camHexUV, camHexUV, 0]
    camCoefsX = [-camHexXY, 0, 0, 0, camHexUV, 0]
    camCoefsY = [0, -camHexXY, 0, camHexUV, 0, 0]
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    m2Coefs = [m2HexXY, m2HexXY, 0, m2HexUV, m2HexUV, 0]
    m2CoefsX = [-m2HexXY, 0, 0, 0, m2HexUV, 0]
    m2CoefsY = [0, -m2HexXY, 0, m2HexUV, 0, 0]

    shiftX = 0
    shiftY = 0
    for i in range(5):
        shiftX += camCoefsX[i] * camShifts[i]
        shiftY += camCoefsY[i] * camShifts[i]
        shiftX += m2CoefsX[i] * m2Shifts[i]
        shiftY += m2CoefsY[i] * m2Shifts[i]
        
    return [shiftX, shiftY]

In [ ]:
test_els = [55, 60, 65, 70]
limit = 1.0
names = ['X', 'Y', 'Z', 'U', 'V']
for test_el in test_els:
    data = np.loadtxt(f"/home/c/cslage/u/Hexapods/data/Hex_Motions_{test_el}_20250715-20250817.txt", skiprows=1)

    expIds = []
    xaxis = []
    camPoss = []
    m2Poss = []
    for n, line in enumerate(data):
        xaxis.append(n)
        expId = int(line[0])
        expIds.append(expId)
        dataId = {'exposure':expId, 'instrument':'LSSTCam'}
        expRecord = getExpRecordFromDataId(butler, dataId)
        (mountErrors, mountData) = calculateMountErrors(expRecord, client)
        poss = []
        for i in range(5):
            val = np.median(mountData.camhexData[f"position{i}"].values)
            if i in [3, 4]:
                val *= 3600.0
            poss.append(val)
        camPoss.append(poss)
        poss = []
        for i in range(5):
            val = np.median(mountData.m2hexData[f"position{i}"].values)
            if i in [3, 4]:
                val *= 3600.0
            poss.append(val)
        m2Poss.append(poss)


    fig, axs = plt.subplots(2, 5, figsize=(12, 5))
    plt.subplots_adjust(wspace=0.8, hspace=1.6)
    plt.suptitle(f"Hexapod state for images with FWHM<{limit}, El={test_el:.1f}", fontsize=18, y=1.0)
    camShifts = np.zeros([5])
    m2Shifts = np.zeros([5])
    for i in range(5):
        cam = [term[i] for term in camPoss]
        shift = np.max(cam) - np.min(cam)
        camShifts[i] = shift
        [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
        image_shift = np.sqrt(shiftX**2 + shiftY**2)
        camShifts[i] = 0.0
        axs[0][i].scatter(xaxis, cam, marker='x')
        axs[0][i].set_xticks(xaxis[::4])
        axs[0][i].set_xticklabels(expIds[::4])
        axs[0][i].tick_params(axis='x', labelrotation=90)
        axs[0][i].set_title(f'Cam{names[i]}\nImage_shift={image_shift:.1f}"')
        if i < 3:
            axs[0][i].set_ylabel("Microns")
        else:
            axs[0][i].set_ylabel("Arcseconds")
        m2 = [term[i] for term in m2Poss]
        shift = np.max(m2) - np.min(m2)
        m2Shifts[i] = shift
        [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
        image_shift = np.sqrt(shiftX**2 + shiftY**2)
        m2Shifts[i] = 0.0
        axs[1][i].scatter(xaxis, m2, marker='x')
        axs[1][i].set_xticks(xaxis[::4])
        axs[1][i].set_xticklabels(expIds[::4])
        axs[1][i].tick_params(axis='x', labelrotation=90)
        axs[1][i].set_title(f'M2{names[i]}\nImage_shift={image_shift:.1f}"')
        if i < 3:
            axs[1][i].set_ylabel("Microns")
        else:
            axs[1][i].set_ylabel("Arcseconds")
    plt.savefig(f"/home/c/cslage/u/Hexapods/data/Hexapod_Stability_{limit}_{test_el}.png")
    plt.clf

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    camPossMed = np.array(camPoss)
    m2PossMed = np.array(m2Poss)
    for i in range(5):
        cam = [term[i] for term in camPoss]
        camMed = np.median(cam)
        camPossMed[:,i] -= camMed
        m2 = [term[i] for term in m2Poss]
        m2Med = np.median(m2)
        m2PossMed[:,i] -= m2Med
    
    shiftXs = []
    shiftYs = []
    for n in range(len(expIds)):
        camShifts = np.zeros([5])
        m2Shifts = np.zeros([5])
        for i in range(5):
            camShifts[i] = camPossMed[n,i]
            m2Shifts[i] = m2PossMed[n,i]
        [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
        shiftXs.append(shiftX)
        shiftYs.append(shiftY)
    
    ax.scatter(shiftXs, shiftYs, marker='x')
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)
    ax.set_xlabel("Arcseconds")
    ax.set_ylabel("Arcseconds")
    ax.set_title(f"Image_shifts for images with FWHM<{limit}, El={test_el:.1f}", fontsize=12, y=1.05)
    plt.savefig(f"/home/c/cslage/u/Hexapods/data/Hexapod_Image_Shift_{limit}_{test_el}.png")


In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12, 5))
plt.subplots_adjust(wspace=0.8, hspace=1.6)
plt.suptitle(f"Hexapod state for images with FWHM<{limit}, El={test_el:.1f}", fontsize=18, y=1.05)
camShifts = np.zeros([5])
m2Shifts = np.zeros([5])
for i in range(5):
    cam = [term[i] for term in camPoss]
    shift = np.max(cam) - np.min(cam)
    camShifts[i] = shift
    [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
    image_shift = np.sqrt(shiftX**2 + shiftY**2)
    camShifts[i] = 0.0
    axs[0][i].scatter(xaxis, cam, marker='x')
    axs[0][i].set_xticks(xaxis[::4])
    axs[0][i].set_xticklabels(expIds[::4])
    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}\nImage_shift={image_shift:.1f}"')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    m2 = [term[i] for term in m2Poss]
    shift = np.max(m2) - np.min(m2)
    m2Shifts[i] = shift
    [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
    image_shift = np.sqrt(shiftX**2 + shiftY**2)
    m2Shifts[i] = 0.0
    axs[1][i].scatter(xaxis, m2, marker='x')
    axs[1][i].set_xticks(xaxis[::4])
    axs[1][i].set_xticklabels(expIds[::4])
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}\nImage_shift={image_shift:.1f}"')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")




In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
camPossMed = np.array(camPoss)
m2PossMed = np.array(m2Poss)
for i in range(5):
    cam = [term[i] for term in camPoss]
    camMed = np.median(cam)
    camPossMed[:,i] -= camMed
    m2 = [term[i] for term in m2Poss]
    m2Med = np.median(m2)
    m2PossMed[:,i] -= m2Med

shiftXs = []
shiftYs = []
for n in range(len(expIds)):
    camShifts = np.zeros([5])
    m2Shifts = np.zeros([5])
    for i in range(5):
        camShifts[i] = camPossMed[n,i]
        m2Shifts[i] = m2PossMed[n,i]
    [shiftX, shiftY] = calculateHexMotion(camShifts, m2Shifts)
    shiftXs.append(shiftX)
    shiftYs.append(shiftY)

ax.scatter(shiftXs, shiftYs, marker='x')
ax.set_xlim(-50, 50)
ax.set_ylim(-50, 50)
ax.set_xlabel("Arcseconds")
ax.set_ylabel("Arcseconds")
ax.set_title(f"Image_shifts for images with FWHM<{limit}, El={test_el:.1f}", fontsize=12, y=1.05)

In [ ]:
test_el = 55
limit = 1.0

data = np.loadtxt(f"/home/c/cslage/u/Hexapods/data/Hex_Motions_{test_el}_20250715-20250817.txt", skiprows=1)
outfilename = f"/home/c/cslage/u/Hexapods/data/Hex_State_{test_el}_20250715-20250817.txt"
outfile = open(outfilename, 'w')
outfile.write("expId              FWHM      Elev.    CamX      CamY      CamZ      CamU      CamV      M2X        M2Y      M2Z       M2U      M2V Compensation ClosedLoop\n")
for n, line in enumerate(data):
    outline = ""
    expId = int(line[0])
    FWHM = float(line[1])
    el = float(line[2])
    outline += f"{expId}  {FWHM:8.2f}  {el:8.2f}  "
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    for i in range(5):
        val = np.median(mountData.camhexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        outline += f"{val:8.1f}  "
    for i in range(5):
        val = np.median(mountData.m2hexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        outline += f"{val:8.1f}  "

    dayObs = expRecord.day_obs
    nextDayObs = calcNextDay(dayObs)
    start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
    end = Time(f"{dayObsIntToString(nextDayObs)}T12:00:00")
    startLoop = getEfdData(client, "lsst.sal.MTAOS.command_startClosedLoop", begin=start, end=end)
    stopLoop = getEfdData(client, "lsst.sal.MTAOS.command_stopClosedLoop", begin=start, end=end)
    comp = getEfdData(client, "lsst.sal.MTHexapod.logevent_compensationMode", begin=start, end=end)
    specified_time = pd.Timestamp(expRecord.timespan.begin.utc.datetime, tz='UTC')
    comp_before = comp[comp.index < specified_time]
    compOn = comp_before['enabled'].iloc[-1]
    outline += f"{compOn}       "
    startLoop_before = startLoop[startLoop.index < specified_time]
    stopLoop_before = stopLoop[stopLoop.index < specified_time]
    if len(stopLoop_before) == 0:
        loopRunning = True
    else:
        if stopLoop_before.index[-1] > startLoop_before.index[-1]:
            loopRunning = False
        else:
            loopRunning = True
    outline += f"{loopRunning}\n"
    outfile.write(outline)
outfile.close()

In [ ]:
expId = 2025072200538
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
camCorrect = getEfdData(client, "lsst.sal.MTAOS.logevent_cameraHexapodCorrection", expRecord=expRecord, prePadding=600)
print(camCorrect.tail(1))
expId = 2025072300281
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
camCorrect = getEfdData(client, "lsst.sal.MTAOS.logevent_cameraHexapodCorrection", expRecord=expRecord, prePadding=600)
print(camCorrect.tail(1))

In [ ]:
names = ['x','y','z','u','v']
outfilename = f"/home/c/cslage/u/Hexapods/data/Hex_Offsets.txt"
outfile = open(outfilename, 'w')
outfile.write("expId             CamX      CamY      CamZ      CamU      CamV      M2X        M2Y      M2Z       M2U      M2V\n")
for expId in [2025072200132, 2025072200538, 2025072300281]:
    outline = f"{expId}  "
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    offset = getEfdData(client, "lsst.sal.MTHexapod.logevent_compensationOffset", expRecord=expRecord, prePadding=600)
    camOffset = offset[offset["salIndex"] == 1]
    for i in range(5):
        outline += f"{camOffset[f"{names[i]}"].iloc[-1]:8.1f}  "
    m2Offset = offset[offset["salIndex"] == 2]
    for i in range(5):
        outline += f"{m2Offset[f"{names[i]}"].iloc[-1]:8.1f}  "
    outline += "\n"
    print(outline)
    outfile.write(outline)
outfile.close()
